## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,avigliana,IT,45.0771,7.3986,58.48,87,99,0.43,overcast clouds
1,1,kodiak,US,57.7900,-152.4072,56.71,41,0,12.66,clear sky
2,2,petropavlovsk-kamchatskiy,RU,53.0452,158.6483,61.39,45,75,11.18,broken clouds
3,3,mataura,NZ,-46.1927,168.8643,58.48,94,100,0.74,overcast clouds
4,4,busselton,AU,-33.6500,115.3333,67.73,62,37,5.48,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,butaritari,KI,3.0707,172.7902,82.40,74,15,16.44,few clouds
7,7,bubaque,GW,11.2833,-15.8333,77.02,71,3,10.87,clear sky
9,9,cairns,AU,-16.9167,145.7667,79.11,83,100,10.36,light rain
10,10,acajutla,SV,13.5928,-89.8275,81.97,76,24,0.98,few clouds
11,11,faanui,PF,-16.4833,-151.7500,82.18,73,9,7.23,clear sky
12,12,mahebourg,MU,-20.4081,57.7000,75.25,94,75,2.30,broken clouds
14,14,vaini,TO,-21.2000,-175.2000,78.96,78,40,9.22,scattered clouds
16,16,bathsheba,BB,13.2167,-59.5167,81.21,78,20,11.50,few clouds
20,20,georgetown,MY,5.4112,100.3354,85.98,75,20,10.36,few clouds
21,21,dunmore town,BS,25.5000,-76.6500,80.08,77,90,9.13,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 6 to 671
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              186 non-null    int64  
 1   City                 186 non-null    object 
 2   Country              186 non-null    object 
 3   Lat                  186 non-null    float64
 4   Lng                  186 non-null    float64
 5   Max Temp             186 non-null    float64
 6   Humidity             186 non-null    int64  
 7   Cloudiness           186 non-null    int64  
 8   Wind Speed           186 non-null    float64
 9   Current Description  186 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 16.0+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
### There are no empty rows!

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,butaritari,KI,82.40,few clouds,3.0707,172.7902,
7,bubaque,GW,77.02,clear sky,11.2833,-15.8333,
9,cairns,AU,79.11,light rain,-16.9167,145.7667,
10,acajutla,SV,81.97,few clouds,13.5928,-89.8275,
11,faanui,PF,82.18,clear sky,-16.4833,-151.7500,
12,mahebourg,MU,75.25,broken clouds,-20.4081,57.7000,
14,vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
16,bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
20,georgetown,MY,85.98,few clouds,5.4112,100.3354,
21,dunmore town,BS,80.08,overcast clouds,25.5000,-76.6500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,butaritari,KI,82.40,few clouds,3.0707,172.7902,Isles Sunset Lodge
7,bubaque,GW,77.02,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"
9,cairns,AU,79.11,light rain,-16.9167,145.7667,Mantra Trilogy Cairns
10,acajutla,SV,81.97,few clouds,13.5928,-89.8275,Laureles
11,faanui,PF,82.18,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,mahebourg,MU,75.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
16,bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
20,georgetown,MY,85.98,few clouds,5.4112,100.3354,Cititel Penang
21,dunmore town,BS,80.08,overcast clouds,25.5000,-76.6500,Coral Sands Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 6 to 671
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 186 non-null    object 
 1   Country              186 non-null    object 
 2   Max Temp             186 non-null    float64
 3   Current Description  186 non-null    object 
 4   Lat                  186 non-null    float64
 5   Lng                  186 non-null    float64
 6   Hotel Name           186 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.7+ KB


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))